<a href="https://colab.research.google.com/github/VitorThome10/Ciencia_de_dados_Python/blob/main/superVisionadoVsRegras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Supervisionado

LSTM

Vader





In [2]:
import pandas as pd
from google.colab import files
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import np_utils

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [3]:
Tweets = pd.read_csv("Tweets2.csv")
Tweets.shape

(24004, 4)

In [4]:
Tweets.head()

,id,local,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [5]:
Tweets.groupby(['sentiment']).size()

sentiment
Irrelevant    4404
Negative      6156
Neutral       6166
Positive      7278
dtype: int64

In [6]:

Tweets.loc[Tweets['sentiment']=='Irrelevant','sentiment'] = 'Neutral'

In [7]:
Tweets = Tweets.dropna(subset=['text'])
Tweets.reset_index(drop=True, inplace=True)

In [8]:
Tweets.shape

(23784, 4)

In [9]:
token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values)

In [10]:
X = token.texts_to_sequences(Tweets['text'].values)
X = pad_sequences(X, padding="post", maxlen=100)

In [11]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['sentiment'])
print(y)

[2 2 2 ... 1 1 1]


In [12]:
y = np_utils.to_categorical(y)
print(y)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.4)
X_test

array([[13, 33, 34, ...,  0,  0,  0],
       [48,  2,  0, ...,  0,  0,  0],
       [ 9, 20, 81, ...,  0,  0,  0],
       ...,
       [76, 12, 33, ...,  0,  0,  0],
       [ 5, 94, 18, ...,  0,  0,  0],
       [74,  5, 89, ...,  0,  0,  0]], dtype=int32)

In [14]:
modelo = Sequential()
modelo.add(Embedding(input_dim= len(token.word_index), output_dim=128, input_length=X.shape[1]))
modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation='tanh',
                recurrent_activation='sigmoid', unroll=False, use_bias=True))
modelo.add(Dense(units=3,activation="softmax"))

In [16]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(modelo.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          2011264   
                                                                 
 spatial_dropout1d (SpatialD  (None, 100, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 3)                 591       
                                                                 
Total params: 2,266,655
Trainable params: 2,266,655
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
modelo.fit(X_train, y_train, epochs=5, batch_size=500,verbose=True)

Epoch 1/5
29/29 [==============================] - 70s 2s/step - loss: 1.0764 - accuracy: 0.4317
Epoch 2/5
29/29 [==============================] - 63s 2s/step - loss: 1.0739 - accuracy: 0.4378
Epoch 3/5
29/29 [==============================] - 58s 2s/step - loss: 1.0739 - accuracy: 0.4378
Epoch 4/5
29/29 [==============================] - 59s 2s/step - loss: 1.0737 - accuracy: 0.4378
Epoch 5/5
29/29 [==============================] - 59s 2s/step - loss: 1.0738 - accuracy: 0.4378


In [18]:
_, accuracy = modelo.evaluate(X_test,y_test)
print("Accuracy: ", accuracy)

298/298 [==============================] - 37s 123ms/step - loss: 1.0732 - accuracy: 0.4404
Accuracy:  0.44040361046791077


In [19]:
#from pandas.core.internals.managers import T
mas = SentimentIntensityAnalyzer()
Tweets['vander_sentiment'] = ''

for y in range(len(Tweets.index)):
  x = mas.polarity_scores(Tweets['text'].iloc[y])
  del x['compound']
  maior = max(x,key=x.get) #neg pos neu
  Tweets.loc[y,'vander_sentiment'] = maior

In [20]:
Tweets.groupby(['sentiment']).size()

sentiment
Negative     6117
Neutral     10437
Positive     7230
dtype: int64

In [21]:
Tweets.loc[Tweets['vander_sentiment']== 'neu' , 'vander_sentiment'] = 'Neutral'
Tweets.loc[Tweets['vander_sentiment']== 'neg' , 'vander_sentiment'] = 'Negative'
Tweets.loc[Tweets['vander_sentiment']== 'pos' , 'vander_sentiment'] = 'Positive'

In [22]:
Tweets.groupby(['vander_sentiment']).size()

vander_sentiment
Negative     1185
Neutral     20660
Positive     1939
dtype: int64

In [23]:
y_pred = Tweets['vander_sentiment']
y_test = Tweets['sentiment']
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 739 5243  135]
 [ 291 9460  686]
 [ 155 5957 1118]]


In [24]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.4758240834174235
